# Create a Database

Create a database that connects Genbank Data to MP3 pathogenicity output. Please see final_project.ipynb for more information. Genbank data can be downloaded form a rast annotation of a genome then run through MP3.  
File structure should be 
- ./
    - create_database.ipynb
    - Groups
        - Group1
            - Group1.faa.Hybrid.result
            - Group1.gbk
        - Group2
            - Group2.faa.Hybrid.result
            - Group2.gbk
    - database.pkl
    
The term groups from the class project. Algorithm may be tweaked for changed filenames and directories

### Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Bio import SeqIO
from collections import Counter

### Extract Data

In [4]:
# Extract genbank data and put into dictionary for recovery later
groups_directory = "./Groups/"
database_directory = "./"
database_name = "salmonella_database.pkl"
groups = [
    1, 3, 4, 5, 6, 7, 
    8, 9, 10, 11, 12, 
    14, 15
]
genbank = {}
for group in groups:
    file_name = "Group" + str(group) + ".gbk"
    file_directory = groups_directory + "Group" + str(group) + "/"
    for seq_record in SeqIO.parse(file_directory + file_name, "genbank"):
        organism = seq_record.annotations["organism"]
        if len(seq_record.features) > 1:
            for feature in seq_record.features[1:]:
                name = feature.qualifiers["db_xref"][0][5:]
                if feature.type != 'CDS':
                    continue
                protein_sequence = feature.qualifiers["translation"][0]
                protein_name = feature.qualifiers["product"][0]
                genbank[name] = [protein_sequence, protein_name, organism]


In [5]:
# Parse mp3 output data
columns = ["Group", "Sr._No.", "Sequence_Name", "Type_of_Pfam_domains", 
           "HMM_Prediction", "SVM_Score", 
           "SVM_prediction", "Hybrid_Prediction", "Assignment", 
           "Sequence", "Product", "Organism"]
data = []
for group in groups:
    file_name = "Group" + str(group) + ".faa.Hybrid.result"
    file_directory = groups_directory + "Group" + str(group) + "/"
    with open(file_directory + file_name, "r") as handle:
        lines = handle.readlines()[1:]
        for line in lines:
            if len(line) == 1:
                continue
            l = line.split('\t')
            if len(l) == 8:
                for i in range(len(l)):
                    l[i] = l[i].strip()
                l[4] = float(l[4])
                l[0] = int(float(l[0]))
                # add in genbank data
                l.extend(genbank[l[1]])
                # add group number field
                l.insert(0, group)
                data.append(l)


### Create Dataframe
We chose to use pandas dataframe for our database because it is easy to implement. In the future we could implement a SQL relational database which would help link our entries. 

In [6]:
df = pd.DataFrame(data=data, columns=columns)

Save to pickle for faster loading without input files

In [7]:
df.to_pickle(database_directory + database_name)

Please see analyze_database.ipynb for querying examples and analysis tools

### Create a sqlite3 Database

It's very easy to transform a pandas dataframe into sqlite3 database. We can simply create a database file, then write the database to that file.

In [25]:
import sqlite3

cn = sqlite3.connect("salmonella.db")
df.to_sql(name="salmonella", con=cn, if_exists="replace")

If we want to execute SQL queries on the database, we first get a cursor, execute a SQL query, then fetch the results.

In [26]:
cur = cn.cursor()
cur.execute("SELECT * FROM salmonella WHERE Hybrid_Prediction == 'Pathogenic';")
cur.fetchall()

[(0,
  1,
  1,
  'fig|6666666.498498.peg.1337',
  'Uncalssified protein',
  '----------',
  -0.090801061,
  'Pathogenic',
  'Pathogenic',
  'S',
  'RQAWRTIALFCVTECFPEDVITDKVEPLTPVYLMTTLMPDVPLTDAGYRQRAKRPVKYPS',
  'hypothetical protein',
  'salmonella sp. Bacteria.'),
 (4,
  1,
  5,
  'fig|6666666.498498.peg.1341',
  'Uncalssified protein',
  '----------',
  1.3877656,
  'Pathogenic',
  'Pathogenic',
  'S',
  'MKNNIEETIGKYLPILMILPLAGLAELASLYSIQALLPKLSEVYNIPLNQVGMILSAEVGFLALAMLFSGTLSDRFGRKPIIFYSLLAGGILTLLCATASSWPMLVVYRALLGIAVSGITAAVTVYISEEVSPALAGIVTGYFIFGNSLGSMSGRVFATLMMEHVSIDTIFFIFGGVLIAMALAVKLFLPTSRQFVPTPSLQLGAVLKGGLEHFKNIRVSLCFVIGFILFGSFTSIFNFLAFYLHRPPYELSYTWIGLIPVSFSLTFFLAPYAARVALNIGSMNALSMLIICMMVGAFLTLIAPSLWVFISGIVLLSVAFFSAHSTVLAWVSSRSPNAKGQATSFYLLCYYSGGAVMGYLNGYLFSWQGWNAIAASCLMMLGIGLFICRFIFAKYEKQPQIKKQSVQESF',
  'Uncharacterized MFS-type transporter STM0328.s',
  'salmonella sp. Bacteria.'),
 (10,
  1,
  11,
  'fig|6666666.498498.peg.1347',
  'Uncalssified protein',
  '----------',

To speed up queries, we can create an index over some column(s) of the database which are commonly accessed. In our case, we probably want to index over "Hybrid_Prediction" since this column tells us the overall pathogenicity result of the MP3 run.

In [27]:
cur.execute("CREATE INDEX path_idx ON salmonella (Hybrid_Prediction);")
cn.commit()

When we're done with the database, we have to commit our queries and close the connection.

In [28]:
cn.commit()
cn.close()